In [1]:
#import gensim
from gensim import corpora, models, similarities, utils
from pprint import pprint   # pretty-printer
import logging
import numpy as np
import pandas as pd
import nltk
import pymysql
import re
import pyLDAvis.gensim
import pyLDAvis

In [2]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [3]:
def call_mysql():
    # pull in data from the reviews table
    con = False
    rows = np.nan
    try:
        con = pymysql.connect(host='localhost', port=3307, user='root', passwd='', db='yelpdata')
        with con:
            cur = con.cursor()
            sql = 'USE yelpdata;'
            cur.execute(sql)
            sql = 'SELECT id,business_id,stars,comment FROM review;' 
            #print(sql)
            cur.execute(sql)
            rows = cur.fetchall()
#             for row in rows:
#                 print(row)
    except pymysql.Error, e:
        print "Error %d: %s" % (e.args[0],e.args[1])
        sys.exit(1)
    finally:
        if con:
            con.close()
    return rows  

In [4]:
rows = call_mysql()

# UNDERSTAND THE OUTPUT OF THE MYSQL
# print(np.shape(rows))#(3616, 4)
# print(type(rows))#tuple
# print(type(rows[:][0]))# still tuple
# print(type(rows[3][3]))# int/str or whatever per the column

#print(rows[0][3])#just the review for the zero-indexed row [0]
# print(rows[1][3])#the review for the one-indexed row [1]

# confusing - not printing all rows of the first column
#print(rows[:][0])#all four columns of the data-row

documents = []#type list
ndocs = np.shape(rows)[0]

#reviews = [[review for review in rows[3]] for row in rows]
for row in rows:
    documents.append(row[3])# row[3] is type string, documents is type list

print(documents[0])# prints one review

# for doc in range(0,ndocs):
#     [review for review in rows[:][0]]# This prints the review from the id in the second []
#     print review#type str

# print(row[3])#just the review
# print(row[:])# a row of data
# print(review)# one review


Adding some more information. Another great thing about him is that he was willing to experiment with me for my fibromyalgia. I brought in a book by a specialist and he was willing to try the protocol. Many doctors would not do that. He won't do anything crazy but will consider something reasonable. He also made himself available when I was having an emergency severe toothearache. He was at an offsite seminar, but his receptionist asked him to call me and he did, prescribed something and I think he even checked in with me later on or I called back. Either way, I got the treatment I needed.For those who think it takes a while to get ahold of him or his receptionistplease keep in mind it is one doctor and one receptionist. As you can see from all of the 5 star reviews, he spends time with his patients. Be a little patient yourself and you will be soon benefit from his great treatment.


In [5]:
def unify_time(word):
    #Assign all possible time values ('9:45','12:05',etc) to one word 'oclock'
    match = re.search(r'\d*'+':'+r'\d*',word)
    if match:
        #print(match.group())
        word = 'oclock'
    return word


    # This also works, better if you want to catch the error and do something, but I don't
    # try: 
    #     match = re.search(r'\d*'+':'+r'\d*',s, word)
    #     test = match.group() 
    #     word = 'oclock'
    # except AttributeError as e: 
    #     print('error in unify_time')

In [6]:
def period_split(word):
    # Split words with a period in the middle - not yet the ideal implementation b/c it returns two words as one
    match = re.search(r'\w*'+'.'+r'\w*',word)
    if match:
        word = word.replace('.',' ')
    return word

In [7]:
def canonicalize(word):    
    bads = ['dr.',"n't","'ve","'re",'wo',"'m"]
    goods = ['dr','not','have','are','will','am']

    try:
        idx = bads.index(word)
        word=goods[idx]
    except ValueError as e:
        word=word
    return word

In [8]:
def unify_pronouns(word):
    """ map many onto one for pronouns - parts of speech"""
    many = ['he',"his","him","she",'her',"hers",'I','i','me','mine','my','you','your',
            'yours','it','its','they','their','theirs','we','us','our','ours']
    few = ['male','male','male','female','female','female','firstper','firstper',
           'firstper','firstper','firstper','secper','secper','secper',
           'neut3rd','neut3rd','neut3rd','neut3rd','neut3rd',
          'coll3rd','coll3rd','coll3rd','coll3rd']
    try:
        idx = many.index(word)
        word=few[idx]
    except ValueError as e:
        word=word
    return word

In [9]:
def clean(word):
    # wrapper function - performs a bunch of functions on a given word
    word = unify_time(word)
    word = unify_pronouns(word)
    word = canonicalize(word)
    word = period_split(word)    
    return word

In [10]:
def numwords():
    #add numbers to common words
    nums = np.arange(0,100)
    ss =''
    ss = ss.join([' '+str(n) for n in nums])
    return ss

In [11]:
def query_SQL():
    con = False
    rows = []
    try:
        con = pymysql.connect(host='localhost', port=3307, user='root', passwd='', db='yelpdata')
        with con:
            cur = con.cursor()
            sql = 'USE yelpdata;'
            cur.execute(sql)
            sql = 'SELECT DISTINCT(yelp_id) FROM business;' 
            #print(sql)
            cur.execute(sql)
            rows = cur.fetchall()
            #print('business id from table: ',rows, type(rows))
    except pymysql.Error, e:
        print "Error %d: %s" % (e.args[0],e.args[1])
        sys.exit(1)
    finally:
        if con:
            con.close()
    return rows  

In [12]:
def ids_to_names():
# Purpose of function is to exclude proper names of doctors - first and last names
# Function will accept list of yelp_ids extracted by mysql from business table
# Function will put together a list of names
# Function will exclude terms like family doctor and pediatrics

    dashed_names=[]#examples = ['tamalpais-pediatrics-novato','julie-doctor-kim']
    yelp_ids = query_SQL()# extracts unique yelp_ids
    for i in range(0,len(yelp_ids)):
        dashed_names.append(yelp_ids[i][0])
    
    def split_append(yelp_id):
        yelp_id = yelp_id.split('-')
        #print(yelp_id)
        return yelp_id

    business_type = ['pediatrics','family','medicine','doctor','institute','center',
                     'practice','md','san','francisco','ucsf','one','medical','group',
                    'sports','fitness','therapy','acupuncture','chiropractic','district']
    names = []
    for name in dashed_names:
        [names.append(l) for l in split_append(name) if l not in business_type]
    return names

In [12]:
# I may end up deleting this.  Now I used regular expressions.  But ids_to_names is not used

# NLTK:  remove common words and tokenize

# Explore this alternative
#from nltk.corpus import stopwords
# for w in stopwords.words():
#     print(w)

common_words = 'for a as was is of the and this that sf to in if , ,, \'s by on the to . .. ...  `` ? & * _ - / "\" 3 2 5'
nums = numwords()
common_words = common_words+nums

#common_names = 'kong lisa chueng cheung dolev laroque caluag khoo usman kim jacoby mccaw simons wong eshima'
common_names = ids_to_names()
#common_words = common_words.join([' '+nums+' '+common_names])
#print(common_words)

stoplist = set(common_words.split())
print(stoplist)

set(['-', '24', '25', '26', '27', '20', '21', '22', '23', '28', '29', '0', '4', '8', '...', 'to', '59', '58', '55', '54', '57', '56', '51', '50', '53', '52', '19', '88', '89', '82', '83', '80', '81', '86', '87', '84', '85', 'for', '/', '3', '7', '?', 'by', '_', 'on', '39', '38', '33', '32', '31', '30', '37', '36', '35', '34', ',', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '..', '&', '*', '.', '2', '6', 'was', '``', 'that', '99', '98', '91', '90', '93', '92', '95', '94', '97', '96', '11', '10', '13', '12', '15', '14', '17', '16', 'this', '18', 'of', 'and', 'is', 'as', 'in', 'if', '48', '49', '46', '47', ',,', '45', '42', '43', '40', '41', '1', '5', '9', "'s", '""', '77', '76', '75', '74', '73', '72', '71', '70', '79', '78', 'a', 'sf', 'the', '44'])


In [13]:
def get_pat_for_tokenizer():
    from nltk.tokenize import regexp_tokenize
    """money and time patterns formatted to fit in nltk RegexpTokenizer object, debugged with s2 below"""
    #s2 = 'it took until 12:30 am to get an appointment and I spent $3.50'
    
    pat = '\w+|\$[\d\.]+|\S+'
    pat2 = '\d+\:[\d]+\s?(pm|am)|\S+'

    pat3 = '('+pat2+')' +'|'+ '('+pat+')'#must be one string argument... | is OR
    #tokenizer = RegexpTokenizer(pat3)#time
    return pat3

In [14]:
# Two ways to clean & tokenize - stoplists, lowercase, regular expressions, time-expressions

# from nltk.corpus import stopwords
# from nltk.tokenize import RegexpTokenizer

#texts: A list of lists; text: list of tokenized words from 1 doc/review
#document = a string

# 1) uses my stoplist - see above
# texts = [[clean(word) for word in nltk.word_tokenize(document.lower()) if word not in stoplist]
#         for document in documents]#review in reviews 

#tokenizer = RegexpTokenizer('\w+|\$[\d\.]+|\S+')#money
#tokenizer = RegexpTokenizer('\d*'+'\:'+'\d*')#time
tokenizer = RegexpTokenizer(get_pat_for_tokenizer())#time & money

# 2) uses nltk functions
texts = [[canonicalize(word) for word in tokenizer.tokenize(document.lower()) if word not in stopwords.words()]
        for document in documents]#review in reviews 

print(texts[:2])

[['adding', 'information.', 'another', 'great', 'thing', 'willing', 'experiment', 'fibromyalgia.', 'brought', 'book', 'specialist', 'willing', 'try', 'protocol.', 'many', 'doctors', 'would', 'that.', "won't", 'anything', 'crazy', 'consider', 'something', 'reasonable.', 'made', 'available', 'emergency', 'severe', 'toothearache.', 'offsite', 'seminar,', 'receptionist', 'asked', 'call', 'did,', 'prescribed', 'something', 'think', 'even', 'checked', 'later', 'called', 'back.', 'either', 'way,', 'got', 'treatment', 'needed.for', 'think', 'takes', 'get', 'ahold', 'receptionistplease', 'keep', 'mind', 'one', 'doctor', 'one', 'receptionist.', 'see', '5', 'star', 'reviews,', 'spends', 'time', 'patients.', 'little', 'patient', 'soon', 'benefit', 'great', 'treatment.'], ["it's", 'tough', 'write', 'review,', "i've", 'patient', 'dr', 'least', '3', 'years', 'now,', 'more.', "i'm", 'huge', 'fan', 'personally,', "he's", 'always', 'given', 'sound', 'advice', 'easy', 'work', 'with,', 'recently.', "he's"

In [15]:
# BAG OF WORDS
dictionary = corpora.Dictionary(texts)# create dictionary object from cleaned texts
#dictionary.save('/Users/kaschbacher/insightproject/yelp/git-yelp/doctor.dict') # store the dictionary, for future reference
print(dictionary)
#print(type(dictionary))#<class 'gensim.corpora.dictionary.Dictionary'>
#print(dictionary.token2id)# very long!

# To merge with another dictionary -  Dictionary.merge_with())# Apply bag of words to reviews & print tokenized dictionary

Dictionary(31837 unique tokens: [u'considered,', u'hanging', u'ovaries,', u'aileen', u'ovaries.']...)


In [27]:
# SERIALIZE & SAVE TO DISK 

# Convert tokenized documents to sparse vectors: -- [(0, 1), (4, 3),...
# new_vec = dictionary.doc2bow(documents[1].lower().split())
# print(new_vec)

corpus = [dictionary.doc2bow(text) for text in texts]
corpus_mm = corpora.MmCorpus.serialize('doctor_bow_for_lda.mm', corpus) # store to disk, for later use

#print(corpus)
# looks like - [[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)... (35, 2)...
# (word order index, frequency of occurence?)

list

In [1]:
# retrain model with more data
# sort documents based on topic assignments
# e.g., look at the top five documents assigned to this topic to get a sense what it's about
# you train model on many doctors' reviews, but when you do the prediction, you can project
# back onto a single doctor
# each review might contain a weightings of topics - 70%(T1)+30%(T2)
# product: personalized report of topics in that Drs reviews

In [18]:
# LDA  (takes awhile to run)
# The constructor estimates Latent Dirichlet Allocation model parameters based on a training corpus 
# id2word is a mapping from word ids (integers) to words (strings). 

# 1) The constructor - obtains LDA model parameters given training corpus
lda = models.ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=10, update_every=0, passes=10, iterations=100)


In [22]:
# SAVE THE OBJECT
# --- note - i tried to do this with gensim, but it wasn't working, and ultimately cpickle did
#from gensim.utils import SaveLoad
#save(fname_or_handle, separately=None, sep_limit=10485760, ignore=frozenset([]), pickle_protocol=2)
#utils.SaveLoad.save(lda, pickle_protocol=2)

import cPickle as pickle
pickle_out = open('lda.p', 'wb')
pickle.dump(lda, pickle_out)
pickle_out.close()

In [33]:
vis_data = pyLDAvis.gensim.prepare(lda, corpus, dictionary)
pyLDAvis.display(vis_data)

In [32]:
pickle.dump(vis_data, open('vis_data.p','wb'))